# LA PREMIERE CELL EST POUR TE MONTRER COMMENT LES TRANSFORMATIONS FONCTIONNENT ET COMMENT L'ENVIRONNEMENT EST MODIFIE

In [ ]:
import gymnasium as gym
from sb3_contrib import RecurrentPPO  # PPO récurrent (LSTM)
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from gymnasium.wrappers import ResizeObservation, GrayscaleObservation
import torch.nn as nn


# référence pour le dictionnaire d'hyperparamètre et des transformations: https://huggingface.co/sb3/ppo_lstm-CarRacing-v0

"""
J'UTILISE DUMMYVECENV ET VECFRAMESTACK CAR LE FrameStackObservation fait bug, chatgpt a proposé cette alternative
Voici sa réponse:
"Le problème vient du fait que Stable-Baselines3 (SB3) et ses algorithmes (y compris RecurrentPPO) s'attendent généralement à un environnement 
vectorisé et utilisent habituellement la classe VecFrameStack pour empiler les observations, plutôt qu'un wrapper Gymnasium standard comme FrameStackObservation

SB3 fonctionne de façon optimale avec un VecEnv (environnement vectorisé), par exemple DummyVecEnv ou SubprocVecEnv

L'empilement d'images se fait alors via VecFrameStack (fournie par SB3) plutôt qu'un wrapper Gymnasium 
(qui n'est pas forcément compatible avec la vérification d'espace d'observation qu'effectue SB3)"


=> C'est simplement l'implementation des environements parallèles de stableBaseline (au lieu de celle de gym utilisée dans le TP)
"""

def make_env():
    def _init():
        env = gym.make("CarRacing-v3", continuous=True, render_mode="rgb_array")
        env = ResizeObservation(env, (64, 64))
        env = GrayscaleObservation(env, keep_dim=True)
        return env
    return _init

print("Loading CarRacing-v3 environment")
env = gym.make("CarRacing-v3", continuous=True, render_mode="rgb_array")
print("Observation space size:", env.observation_space.shape)
env = ResizeObservation(env, (64, 64)) # 64 from the dict
print("Observation space resize:", env.observation_space.shape)
print("Observation space:", env.observation_space)
env = GrayscaleObservation(env, keep_dim = True)
print("Observation space after gray scaling:", env.observation_space)

n_envs = 8
env = DummyVecEnv([make_env() for _ in range(n_envs)])
env = VecFrameStack(env, n_stack=2)
print(env.observation_space)

Loading CarRacing-v3 environment
Observation space size: (96, 96, 3)
Observation space resize: (64, 64, 3)
Observation space: Box(0, 255, (64, 64, 3), uint8)
Observation space after gray scaling: Box(0, 255, (64, 64, 1), uint8)
Observation space size after frame stacking: Box(0, 255, (64, 64, 1), uint8)
Box(0, 255, (64, 64, 2), uint8)


In [3]:
import gymnasium as gym
from sb3_contrib import RecurrentPPO  # PPO récurrent (LSTM)
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecNormalize
from stable_baselines3.common.callbacks import BaseCallback
from gymnasium.wrappers import ResizeObservation, GrayscaleObservation
import torch.nn as nn
import wandb
from wandb.integration.sb3 import WandbCallback
import yaml

# référence pour le dictionnaire d'hyperparamètre et des transformations: https://huggingface.co/sb3/ppo_lstm-CarRacing-v0

"""
DANS LE DICT IL Y A ENCORE

https://rl-baselines3-zoo.readthedocs.io/en/master/guide/config.html
https://stable-baselines3.readthedocs.io/en/master/common/logger.html => episodic len / episodic rew

('learning_rate', 'lin_1e-4') => decrease linéaire du lr je suppose (cfr TP5)
('normalize', "{'norm_obs': False, 'norm_reward': True}") => Normalization des reward mais pas des observations
('normalize_kwargs', {'norm_obs': False, 'norm_reward': False})] => à investiguer, c'est chelou car par cohérant avec le champ précédent
"""

# Charger la configuration YAML depuis "config.yaml"
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extraire les hyperparamètres et la configuration de la policy
hyperparams = config["hyperparams"]
policy_kwargs = config["policy_kwargs"]
transformation = config["transformation"]
total_timesteps = config["total_timesteps"]

resize_obs_shape = tuple(transformation.get("resize_observation", None))
grayscale_params = transformation.get("grayscale_observation", None)
n_envs = transformation.get("n_envs", 1)
frame_stack = transformation.get("frame_stack", 1)
normalize_params = transformation.get("normalize", None)

# Si learning_rate est défini comme un dictionnaire dans le YAML,
# on crée une fonction de planification linéaire
if isinstance(hyperparams.get("learning_rate"), dict):
    initial_lr = hyperparams["learning_rate"].get("initial", 1e-4)
    def linear_schedule(progress_remaining):
        return progress_remaining * initial_lr
    hyperparams["learning_rate"] = linear_schedule

# Convertir la chaîne de caractère de activation_fn en fonction réelle
if "activation_fn" in policy_kwargs:
    activation_str = policy_kwargs["activation_fn"]
    policy_kwargs["activation_fn"] = getattr(nn, activation_str)

# Ajouter policy_kwargs dans hyperparams pour le passer au modèle
hyperparams["policy_kwargs"] = policy_kwargs

# Initialisation de wandb
run = wandb.init(
    entity="Rl2025-project",
    project="RL Project",
    name="First_full_test_with_yaml_config",
    config=hyperparams,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,       # auto-upload des vidéos de l'agent
    # save_code=True,       # optionnel
)

# Fonction de création d'environnement
def make_env():
    def _init():
        env = gym.make("CarRacing-v3", continuous=True, lap_complete_percent=0.95, domain_randomize=False, render_mode="rgb_array")
        if resize_obs_shape is not None:
            env = ResizeObservation(env, resize_obs_shape)
        if grayscale_params is not None:
            env = GrayscaleObservation(env, **grayscale_params)
        env = Monitor(env)
        return env
    return _init

print("Loading CarRacing-v3 environment")
env = DummyVecEnv([make_env() for _ in range(n_envs)])
env = VecFrameStack(env, n_stack=frame_stack)
if normalize_params is not None:
    env = VecNormalize(env, **normalize_params)

print("Doing PPOLSTM")
model = RecurrentPPO("CnnLstmPolicy", env, verbose=1, stats_window_size=1, tensorboard_log=f"runs/{run.id}", **hyperparams)

model.learn(
    total_timesteps=total_timesteps,
    callback=WandbCallback(
        gradient_save_freq=100,
        verbose=2,
    ),
)

wandb.finish()
# Sauvegarde du modèle final
model.save("q2_final")

Loading CarRacing-v3 environment
Doing PPOLSTM
Using cpu device
Wrapping the env in a VecTransposeImage.


wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Logging to runs/7iq1mgm4\RecurrentPPO_1
-----------------------------
| time/              |      |
|    fps             | 61   |
|    iterations      | 1    |
|    time_elapsed    | 66   |
|    total_timesteps | 4096 |
-----------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1e+03       |
|    ep_rew_mean          | -18.1       |
| time/                   |             |
|    fps                  | 50          |
|    iterations           | 2           |
|    time_elapsed         | 163         |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.113065444 |
|    clip_fraction        | 0.299       |
|    clip_range           | 0.2         |
|    entropy_loss         | 4.01        |
|    explained_variance   | -0.000114   |
|    learning_rate        | 9.98e-05    |
|    loss                 | 0.318       |
|    n_updates            | 10      

KeyboardInterrupt: 

In [5]:
wandb.finish()

In [1]:

import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
import wandb
from wandb.integration.sb3 import WandbCallback


config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 25000,
    "env_name": "CartPole-v1",
}
run = wandb.init(
    project="RL Project",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
)


def make_env():
    env = gym.make(config["env_name"])
    env = Monitor(env)  # record stats such as returns
    return env


env = DummyVecEnv([make_env])

model = PPO(config["policy_type"], env, verbose=1, tensorboard_log=f"runs/{run.id}")
model.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=100,
        verbose=2,
    ),
)
run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: flo230702 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/florenthervers/miniconda3/envs/car_racing_env/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device
Logging to runs/f7vigkl3/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.5     |
|    ep_rew_mean     | 22.5     |
| time/              |          |
|    fps             | 1313     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 27.9        |
|    ep_rew_mean          | 27.9        |
| time/                   |             |
|    fps                  | 967         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008340905 |
|    clip_fraction        | 0.111       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance 

global_step,▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇███
rollout/ep_len_mean,▁▁▁▂▂▃▄▄▅▆▆▇█
rollout/ep_rew_mean,▁▁▁▂▂▃▄▄▅▆▆▇█
time/fps,█▄▃▂▂▂▂▁▁▁▁▁▁
train/approx_kl,▆▇▆█▆▃▄▃▅▁▁█
train/clip_fraction,█▅▅▆▄▂▄▂▄▁▁▅
train/clip_range,▁▁▁▁▁▁▁▁▁▁▁▁
train/entropy_loss,▁▂▃▄▅▅▅▆▆▇▇█
train/explained_variance,▁▂▃▄▄▆▇▅▇▁▇█
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▃▅▇▅▇▃▃▅▅█▅▁
